In [1]:
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix
import scipy.sparse as sp
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import string
from string import digits
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


[nltk_data] Downloading package stopwords to /Users/arodi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/arodi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/arodi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#separate training data and labels
#print(os.path)
train_data_path='/Users/arodi/Desktop/Fall-2021/CMPE-255/project/CMPE-255_S3-Group10/C50/C50train/'
#change the data pathname while running based on location
author_list=os.listdir(train_data_path)#lists all the authors in path 'train_data_path'
train_labels=[]
author_text_tuple=[]
train_set=[]
authindex=0
#print('l',l)
for p in author_list:
    #print(p)
    if (not p.startswith('.')):
        file_list=os.listdir(train_data_path+p)#lists all the files in the path "train_data_path+p"
        #file_list.remove('.ipynb_checkpoints') # remove .ipynb_checkpoints if data is present  in jupyter notebook
        #file_list.remove('.DS_store') # remove .DS_store files if data is prsent in local drive
        path2_each_author=train_data_path+p#path for every author folder
        #print('file_list',file_list) #print all text files names for each author
        
        for author in author_list:
            if (not author.startswith('.')):  # this condition is to keep out all hidden files
                if author not in train_labels:
                    train_labels.append(author)
        writings = ''
        for file in range(len(file_list)):
            #print(path2_each_author+'/'+file)
            with open (path2_each_author+'/'+file_list[file],'r') as fp:
                writings+=fp.read()
                #author_text_tuple.append((author_list[authindex],file_list[file],(fp.read().splitlines())))
        train_set.append(writings)
        #authindex+=1
print("Number of train labels",len(train_labels))
print("Train labels",train_labels)#list of authors which are class labels
print("-"*50)
#print(author_text_tuple)# list of tuples with filename,content
#print(train_set)# print combined text from each author

        
#separate test data and labels
#print(os.path)
test_data_path='/Users/arodi/Desktop/Fall-2021/CMPE-255/project/CMPE-255_S3-Group10/C50/C50test/'
author_list=os.listdir(test_data_path)#lists all the authors in path 'train_data_path'
test_labels=[]
author_text_tuple=[]
test_set=[]
authindex=0
#print('l',l)
for p in author_list:
    #print(p)
    if (not p.startswith('.')):
        file_list=os.listdir(test_data_path+p)#lists all the files in the path "test_data_path+p"
        #file_list.remove('.ipynb_checkpoints') # remove .ipynb_checkpoints if data is present  in jupyter notebook
        #file_list.remove('.DS_store') # remove .DS_store files if data is prsent in local drive
        path2_each_author=test_data_path+p#path for every author folder
        #print('file_list',file_list)
        
        for author in author_list:
            if (not author.startswith('.')):  # this condition is to keep out all hidden files
                if author not in test_labels:
                    test_labels.append(author)
        writings = ''
        for file in range(len(file_list)):
            #print(path2_each_author+'/'+file)
            with open (path2_each_author+'/'+file_list[file],'r') as fp:
                writings+=fp.read()
                #author_text_tuple.append((author_list[authindex],file_list[file],(fp.read().splitlines())))
        test_set.append(writings)
        #authindex+=1
print("Number of test labels:",len(train_labels))
print("Test labels",train_labels)#list of authors which are class labels
print("-"*50)
#print(author_text_tuple)# list of tuples with filename,content
#print(test_set)# print combined text from each author 

# some data preprocessing todo



Number of train labels 50
Train labels ['RobinSidel', 'LynnleyBrowning', 'KouroshKarimkhany', 'MichaelConnor', 'JoeOrtiz', 'EricAuchard', 'AaronPressman', 'SimonCowell', "LynneO'Donnell", 'EdnaFernandes', 'KevinMorrison', 'SamuelPerry', 'PatriciaCommins', 'JohnMastrini', 'JanLopatka', 'KevinDrawbaugh', 'KarlPenhaul', 'MartinWolk', 'ScottHillis', 'DavidLawder', 'FumikoFujisaki', 'MarcelMichelson', 'NickLouth', 'DarrenSchuettler', 'WilliamKazer', 'TanEeLyn', 'PierreTran', 'HeatherScoffield', 'MureDickie', 'RogerFillion', 'JimGilchrist', 'BradDorfman', 'AlanCrosby', 'JonathanBirt', 'BenjaminKangLim', 'TheresePoletti', 'KeithWeir', 'JoWinterbottom', 'MarkBendeich', 'JaneMacartney', 'MatthewBunce', 'ToddNissen', 'PeterHumphrey', 'TimFarrand', 'SarahDavison', 'GrahamEarnshaw', 'BernardHickey', 'KirstinRidley', 'AlexanderSmith', 'LydiaZajc']
--------------------------------------------------
Number of test labels: 50
Test labels ['RobinSidel', 'LynnleyBrowning', 'KouroshKarimkhany', 'MichaelC